In [ ]:
import wget

In [ ]:
# download file from url
wget.download('https://raw.githubusercontent.com/Denis2054/Transformers-for-NLP-2nd-Edition/master/Chapter04/kant.txt', 'F:/datasets/pre-trained-roberta/kant.txt')

In [ ]:
from pathlib import Path

In [ ]:
from tokenizers import ByteLevelBPETokenizer

In [ ]:
path = 'F:/datasets/pre-trained-roberta/kant.txt'

In [ ]:
print(str(Path(".").glob("D:/datasets/pre-trained-roberta/*.txt")))

In [ ]:
tokenizer = ByteLevelBPETokenizer()

In [ ]:
tokenizer.train(files=path, vocab_size=52_000, min_frequency=2, special_tokens=[ "<s>", "<pad>", "</s>", "<unk>", "<mask>" ])

In [ ]:
tokenizer.save_model("F:\datasets\pre-trained-roberta\KantaiBERT")

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer

In [ ]:
from tokenizers.processors import BertProcessing

In [ ]:
tokenizer = ByteLevelBPETokenizer('F:/datasets/pre-trained-roberta/KantaiBERT/vocab.json', 'F:/datasets/pre-trained-roberta/KantaiBERT/merges.txt')

In [ ]:
# let's test the tokenizer
tokenizer.encode("Hi, everyone! How you are enjoying this presentation!").tokens

In [ ]:
# see number of tokens
tokenizer.encode("Hello, everyone! How you are enjoying this presentation!")

In [ ]:
# add start and end tokens
tokenizer._tokenizer.post_processor = BertProcessing(("</s>", tokenizer.token_to_id("</s>")), ("<s>", tokenizer.token_to_id("<s>")),)


In [ ]:
tokenizer.enable_truncation(max_length=512)

In [ ]:
# lets encode the sentence
tokenizer.encode("Hello, everyone! How you are enjoying this presentation!")

In [ ]:
tokenizer.encode("Hello, everyone! How you are enjoying this presentation!").tokens

In [ ]:
import torch
# check specs of available GPU
torch.cuda.get_device_name(0)

In [ ]:
# Compute Unified Device Architecture (CUDA) was developed by NVIDIA to use the parallel computing power of GPUs for general purpose computing.
torch.cuda.is_available()


In [ ]:
from transformers import RobertaConfig


In [ ]:
config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaTokenizer

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('F:/datasets/pre-trained-roberta/KantaiBERT', max_len=512)

In [ ]:
from transformers import RobertaForMaskedLM

In [ ]:
model = RobertaForMaskedLM(config=config)

In [ ]:
print(model.config)

In [ ]:
# build dataset
from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='F:/datasets/pre-trained-roberta/kant.txt',
    block_size=128,
)

In [ ]:
# COLATOR
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
# trainer
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir='F:/datasets/pre-trained-roberta/KantaiBERT',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
# pretraining the model

trainer.train()

In [ ]:
# save model
trainer.save_model('F:/datasets/pre-trained-roberta/KantaiBERT')

In [1]:
# load model
from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM.from_pretrained('F:/datasets/pre-trained-roberta/KantaiBERT')


In [2]:
# fill mask pipeline
from transformers import pipeline
fill_mask = pipeline(
    "fill-mask",
    model="F:/datasets/pre-trained-roberta/KantaiBERT", 
    tokenizer="F:/datasets/pre-trained-roberta/KantaiBERT"
)

In [ ]:
# ask our model to think like immanuel Kant
fill_mask("Immanuel Kant was a German philosopher who is a central figure in modern philosophy. In his writings, the term \"metaphysics\" refers to: <mask>")

In [3]:
fill_mask("Human thinking is a <mask> phenomenon.")

[{'score': 0.032270532101392746,
  'token': 610,
  'token_str': ' practical',
  'sequence': 'Human thinking is a practical phenomenon.'},
 {'score': 0.021122438833117485,
  'token': 468,
  'token_str': ' pure',
  'sequence': 'Human thinking is a pure phenomenon.'},
 {'score': 0.019380025565624237,
  'token': 589,
  'token_str': ' empirical',
  'sequence': 'Human thinking is a empirical phenomenon.'},
 {'score': 0.017984643578529358,
  'token': 483,
  'token_str': ' mere',
  'sequence': 'Human thinking is a mere phenomenon.'},
 {'score': 0.017538659274578094,
  'token': 758,
  'token_str': ' transcendental',
  'sequence': 'Human thinking is a transcendental phenomenon.'}]

In [ ]:
fill_mask("The human mind is a <mask> phenomenon.")